In [ ]:
# This is necessary to recognize the modules
import os
import sys
from decimal import Decimal
import warnings

warnings.filterwarnings("ignore")

root_path = os.path.abspath(os.path.join(os.getcwd(), '../..'))
sys.path.append(root_path)

In [ ]:
from core.backtesting import BacktestingEngine

backtesting = BacktestingEngine(root_path=root_path, load_cached_data=False)

In [ ]:
from hummingbot.strategy_v2.executors.position_executor.data_types import TrailingStop
from controllers.directional_trading.raj_reversion import RAJReversionControllerConfig
import datetime
from decimal import Decimal

# Controller configuration
connector_name = "binance_perpetual"
trading_pair = "DOT-USDT"
interval = "1h"
# Mean Reversion Strategy Parameters
close_alma_length = 80  # Length input 
close_alma_offset = 0.85  # Offset input
close_alma_sigma = 16  # Sigma input

# Pivot Parameters
left = 7  # Left pivot input
right = 7  # Right pivot input
array_percent = 86  # Percentile threshold
percentile_rolling_window = 300

# ALMA over mean Parameters
diff_alma_length = 9  # Second ALMA length
diff_alma_offset = 0.85  # Second ALMA offset  
diff_alma_sigma = 16  # Second ALMA sigma

total_amount_quote = 1000
max_executors_per_side = 1
take_profit = 0.04
stop_loss = 0.02
trailing_stop_activation_price = 0.01
trailing_stop_trailing_delta = 0.005
time_limit = 60 * 60 * 24 * 2
cooldown_time = 60 * 15

start = int(datetime.datetime(2024, 10, 12).timestamp())
end = int(datetime.datetime(2024, 12, 12).timestamp())


# Creating the instance of the configuration and the controller
config = RAJReversionControllerConfig(
    connector_name=connector_name,
    trading_pair=trading_pair,
    interval=interval,
    close_alma_length=close_alma_length,
    close_alma_offset=close_alma_offset,
    close_alma_sigma=close_alma_sigma,
    pivot_left=left,
    pivot_right=right,
    array_percent=array_percent,
    percentile_rolling_window=percentile_rolling_window,
    diff_alma_length=diff_alma_length,
    diff_alma_offset=diff_alma_offset,
    diff_alma_sigma=diff_alma_sigma,
    total_amount_quote=Decimal(total_amount_quote),
    take_profit=Decimal(take_profit),
    stop_loss=Decimal(stop_loss),
    trailing_stop=TrailingStop(activation_price=Decimal(trailing_stop_activation_price), trailing_delta=Decimal(trailing_stop_trailing_delta)),
    time_limit=time_limit,
    max_executors_per_side=max_executors_per_side,
    cooldown_time=cooldown_time,
)

In [ ]:
backtesting_result = await backtesting.run_backtesting(config, start, end, "1m")

In [ ]:
# Let's see what is inside the backtesting results
print(backtesting_result.get_results_summary())
backtesting_result.get_backtesting_figure()

In [ ]:
# 2. The executors dataframe: this is the dataframe that contains the information of the orders that were executed
import pandas as pd

executors_df = backtesting_result.executors_df
executors_df.head()

### Backtesting Analysis

### Scatter of PNL per Trade
This bar chart illustrates the PNL for each individual trade. Positive PNLs are shown in green and negative PNLs in red, providing a clear view of profitable vs. unprofitable trades.


In [ ]:
import plotly.express as px

# Create a new column for profitability
executors_df['profitable'] = executors_df['net_pnl_quote'] > 0

# Create the scatter plot
fig = px.scatter(
    executors_df,
    x="timestamp",
    y='net_pnl_quote',
    title='PNL per Trade',
    color='profitable',
    color_discrete_map={True: 'green', False: 'red'},
    labels={'timestamp': 'Timestamp', 'net_pnl_quote': 'Net PNL (Quote)'},
    hover_data=['filled_amount_quote', 'side']
)

# Customize the layout
fig.update_layout(
    xaxis_title="Timestamp",
    yaxis_title="Net PNL (Quote)",
    legend_title="Profitable",
    font=dict(size=12, color="white"),
    showlegend=False,
    plot_bgcolor='rgba(0,0,0,0.8)',  # Dark background
    paper_bgcolor='rgba(0,0,0,0.8)',  # Dark background for the entire plot area
    xaxis=dict(gridcolor="gray"),
    yaxis=dict(gridcolor="gray")
)

# Add a horizontal line at y=0 to clearly separate profits and losses
fig.add_hline(y=0, line_dash="dash", line_color="lightgray")

# Show the plot
fig.show()

### Histogram of PNL Distribution
The histogram displays the distribution of PNL values across all trades. It helps in understanding the frequency and range of profit and loss outcomes.


In [ ]:
fig = px.histogram(executors_df, x='net_pnl_quote', title='PNL Distribution')
fig.show()


# Conclusion
We can see that the indicator has potential to bring good signals to trade and might be interesting to see how we can design a market maker that shifts the mid price based on this indicator.
A lot of the short signals are wrong but if we zoom in into the loss signals we can see that the losses are not that big and the wins are bigger and if we had implemented the trailing stop feature probably a lot of them are going to be profits.

# Next steps
- Filter only the loss signals and understand what you can do to prevent them
- Try different configuration values for the indicator
- Test in multiple markets, pick mature markets like BTC-USDT or ETH-USDT and also volatile markets like DOGE-USDT or SHIB-USDT